In [1]:
import json

from copy import deepcopy
from random import choice, shuffle

from entities.clue import Clue
from entities.room import Room
from entities.weapon import Weapon
from entities.suspicious import Suspicious

from entities.agent import Agent

from entities.crime import Crime

from utils.clue_utils import classify_clues

#### Load raw data

In [2]:
with open('data/cards.json', 'r') as file:
    cards = json.load(file)

In [3]:
cards

{'weapons': ['Candlestick', 'Knife', 'Pipe', 'Revolver', 'Rope', 'Wrench'],
 'suspicious': ['Miss Scarlett',
  'Mr. Green',
  'Colonel Mustard',
  'Professor Plum',
  'Mrs. Peacock',
  'Mrs. White'],
 'rooms': ['Kitchen',
  'Ballroom',
  'Conservatory',
  'Dining Room',
  'Billiard Room',
  'Library',
  'Lounge',
  'Hall',
  'Study']}

#### Build Cards

In [4]:
clues = []
id_counter = 0
for key, values in cards.items():
    if key == 'weapons':
        for value in values:
            clues.append(Weapon(name=value, clue_id=id_counter))
            id_counter += 1
    if key == 'rooms':
        for value in values:
            clues.append(Room(name=value, clue_id=id_counter))
            id_counter += 1
    if key == 'suspicious':
        for value in values:
            clues.append(Suspicious(name=value, clue_id=id_counter))
            id_counter += 1

In [5]:
raw_clues = deepcopy(clues)

In [6]:
raw_clues

#### Pick Guilty

In [7]:
weapons, rooms, suspicious = classify_clues(clues=clues)

In [8]:
crime = Crime(weapon=choice(weapons), 
              room=choice(rooms), 
              suspicious=choice(suspicious))

In [9]:
crime.to_dict(verbose=1)

{'suspicious': 'Colonel Mustard',
 'weapon': 'Candlestick',
 'room': 'Billiard Room'}

In [10]:
clues.remove(crime.room)
clues.remove(crime.suspicious)
clues.remove(crime.weapon)

#### Split Cards into Groups

In [11]:
shuffle(clues)

In [13]:
clues

In [14]:
players = 6
number_of_cards = int(len(clues)/players)
grouped_cards = [
    clues[i * number_of_cards:(i + 1) * number_of_cards] 
    for i in range((len(clues) + number_of_cards - 1) // number_of_cards )
]

In [15]:
grouped_cards

[[<entities.room.Room at 0x7f1b50f18dd0>,

#### Build Agents

In [16]:
player_names = ['Dick tracy', 'Shelock', 'Watson', 'Inspector Gatget', 'Batman', 'Tintin']

In [17]:
agents = [
    Agent(name=name, clues=deepcopy(raw_clues), own_clues=cards) 
    for name, cards in zip(player_names, grouped_cards)
]

#### Run Turn

In [18]:
guess = agents[0].guess()

In [19]:
guess.to_dict(verbose=1)

{'suspicious': 'Miss Scarlett', 'weapon': 'Candlestick', 'room': 'Kitchen'}

In [20]:
response = {}
for index in range(1,6):
    response.update(agents[index].response(question=guess))

In [21]:
response

{<entities.room.Room at 0x7f1b3bc24950>: 1,
 <entities.suspicious.Suspicious at 0x7f1b3bc24690>: 1}

In [22]:
if not len(response):
    print(crime.to_dict(verbose=1))
    print(guess.to_dict(verbose=1))
    assert crime.to_dict(verbose=1) == guess.to_dict(verbose=1)

In [24]:
agents[0].process_response(response=response)

In [25]:
agents[0].report()

{'Kitchen': 1,
 'Ballroom': 0.1111111111111111,
 'Conservatory': 0.1111111111111111,
 'Dining Room': 0.1111111111111111,
 'Billiard Room': 0.1111111111111111,
 'Library': 0.1111111111111111,
 'Lounge': 0.1111111111111111,
 'Hall': 0.1111111111111111,
 'Study': 1,
 'Candlestick': 0.16666666666666666,
 'Knife': 1,
 'Pipe': 0.16666666666666666,
 'Revolver': 0.16666666666666666,
 'Rope': 0.16666666666666666,
 'Wrench': 0.16666666666666666,
 'Miss Scarlett': 1,
 'Mr. Green': 0.16666666666666666,
 'Colonel Mustard': 0.16666666666666666,
 'Professor Plum': 1,
 'Mrs. Peacock': 0.16666666666666666,
 'Mrs. White': 0.16666666666666666}